In [15]:
include("MiniCollectiveSpins.jl")
using PyPlot
using Statistics
using JLD2
using OrdinaryDiffEq
import PhysicalConstants.CODATA2018: c_0
using Unitful
using ProgressMeter
using NonlinearSolve, ForwardDiff
using SteadyStateDiffEq 
using BenchmarkTools

In [16]:
""" Prepare the initial vector u0 """
function u0_CFunction(phi_array, theta_array, op_list)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            for op in op_list[i]
                j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                if parse(Int, string(op)[1:2]) == 22
                    u0[i] *= cos(theta_array[j]/2)^2
                elseif parse(Int, string(op)[1:2]) == 21
                    u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                elseif parse(Int, string(op)[1:2]) == 12
                    u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                else
                    println(op)
                end
            end
        end
    end
    return u0
end

u0_CFunction

In [17]:
""" Create a random distribution, save it, computes the corresponding parameters an return the stationnary state. 
If compute_t_evolution, compute the whole evolution, else only the stationnary state. """
function solve_random_distrib(chunk, f, op_list, compute_t_evolution, N, n, d0_lb, nbr_max_iteration)
    popup_ss, popup_t = [], []
    nbr_error_ss, nbr_error_t = [], []

    for i in chunk
        # Compute distribution
        L = (N/n)^(1/3) # Change the volume to keep the density cste
        r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]

        # Choose a distribution where the minimum distance between the atoms is bigger than d0_min
        while min_r0(r0) < d0_lb
            r0 = [[rand(Float64)*L, rand(Float64)*L, rand(Float64)*L] for i in 1:N]
        end

        # Save the atoms position for comparison with QuantumOptics
        @save "r0/r0_N_$(N)_r_$i.jdl2" r0 L

        # Compute the parameters
        system = SpinCollection(r0, e, gammas=1.)
        Ω_CS = OmegaMatrix(system)
        Γ_CS = GammaMatrix(system)
        Γij_ = [Γ_CS[i, j] for i = 1:N for j=1:N]
        Ωij_ = [Ω_CS[i, j] for i = 1:N for j=1:N if i≠j]
        exp_RO_ = [exp(1im*r0[i]'kl) for i = N:-1:1] # We go in the decreasing direction to avoid exp_RO(10) being replace by exp_RO(1)0
        conj_exp_RO_ = [exp(-1im*r0[i]'kl) for i = N:-1:1]
        p0 = ComplexF64.([Γij_; Ωij_; exp_RO_; conj_exp_RO_; Ω_RO/2])

        ## Compute stationnary state ##
        # Prepare the final state guess
        θ_f = range(π, π/2, nbr_max_iteration)
        phi_array_f, theta_array_f = zeros(N), ones(N)*θ_f[1]
        uf = u0_CFunction(phi_array_f, theta_array_f, op_list)

        ## Computation steady state ##
        prob_ss = NonlinearProblem(f, uf, p0)
        sol_ss = solve(prob_ss, NewtonRaphsonBatched(); linsolve = :qr, lower_bounds = fill(-1.0, length(uf)), upper_bounds = fill(1.0, length(uf))) # ; autodiff = ForwardDiff
        print(SciMLBase.successful_retcode(sol_ss))

        nbr_iteration = 2
        while ~SciMLBase.successful_retcode(sol_ss) | nbr_iteration < nbr_max_iteration
            phi_array_f, theta_array_f = zeros(N), ones(N)*θ_f[nbr_iteration] # We start from all the atoms in the GS
            uf = u0_CFunction(phi_array_f, theta_array_f, op_list)
            prob_ss = NonlinearProblem(f, uf, p0)
            sol_ss = solve(prob_ss, NewtonRaphsonBatched(); linsolve = :qr, lower_bounds = fill(-1.0, length(uf)), upper_bounds = fill(1.0, length(uf)))
            nbr_iteration += 1
        end
        push!(popup_ss, sum(real(sol_ss[1:N])))


        # ## Compute evolution ##
        if compute_t_evolution
            # Prepare the initial state
            phi_array_0, theta_array_0 = zeros(N), ones(N)*π # We start from all the atoms in the GS
            u0 = u0_CFunction(phi_array_0, theta_array_0, op_list)
            
            # Load the functions
            fsolve(du, u, p, t) = f(du, u, p0)
            
            prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (T[1], T[end]))

            sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5(), saveat=T;
                        reltol=1.0e-6,
                        abstol=1.0e-8)

            if SciMLBase.successful_retcode(sol)
                push!(popup_t, [sum(real(sol.u[i][1:N])) for i=1:length(T)])
            else
                push!(nbr_error_t, i)
            end
        end
    end
    return popup_ss, popup_t, nbr_error_ss, nbr_error_t 
end

""" Return the minimum distance of a distribution of atoms r0 """
function min_r0(r0)
    N = length(r0)
    d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
    for j in 1:N
        for k = 1:N
            d0[j, k] = norm(r0[j]-r0[k])
        end
    end
    return minimum(d0[d0 .> 0])
end

min_r0

### Define the system

In [18]:
# Nbr of particles
N = 4
r = 10 # Nbr of repetitions

# Normalisation parameters
λ = 421e-9
γ = 32.7e6 # In Hz

# Physical values
ω0 = (2π*ustrip(c_0)/λ)
ωl = ω0
kl = [ustrip(c_0)/ωl, 0, 0] # Laser along x
Ω_RO = 1e7 # Taken from Barbut arXiv:2412.02541v1

# Fixed density
n0 = 2e3 # atoms per unit of volume (already normalized)
d0_lb = 2e-10 # Minimum distance between the atoms (lower boundary) in m

# Normalization
ω0 = ω0 / γ
ωl = ωl / γ
kl = kl * λ
Ω_RO = Ω_RO / γ
d0_lb = d0_lb/λ

# Quantization axis along z
e = [0, 0, 1.]

# Integration parameter
tstep = 0.1
T = [0:tstep:100;] # Normalised time
nbr_max_iteration = 10; # Max iteration per distribution to find the SS

### Compute stationnary state for r repetitions

In [19]:
# Create the directories
if !isdir("r0")
    mkdir("r0")
end
if !isdir("Images_distribution")
    mkdir("Images_distribution")
end
if !isdir("solutions")
    mkdir("solutions")
end
nothing 

In [20]:
# Prepare the function
libpath ="libs/liballfuncs_$N.dll"
f(du, u, params) = ccall(("diffeqf", libpath), Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}, Ptr{ComplexF64}), du, u, params)

f (generic function with 1 method)

In [21]:
@load "op_list/op_list_$N.jdl2" op_list
list_r = 1:r
chunks = Iterators.partition(list_r, cld(length(list_r), Threads.nthreads()))
tasks = map(chunks) do chunk # Split the different distributions into chuncks solved on each core
    Threads.@spawn solve_random_distrib(chunk, f, op_list, true, N, n0, d0_lb, nbr_max_iteration)
end

# Gather the data from the different threads
sol_tasks = fetch.(tasks)
popup_ss = vcat([s[1] for s in sol_tasks]...)
popup_t = vcat([s[2] for s in sol_tasks]...)
nbr_error_ss = vcat([s[3] for s in sol_tasks]...)
nbr_error_t = vcat([s[3] for s in sol_tasks]...)
# print("Nbr errors = "*string(length(nbr_error)))
nbr_error_ss

LoadError: TaskFailedException

[91m    nested task error: [39mUndefVarError: `NewtonRaphsonBatched` not defined in `Main`
    Suggestion: check for spelling errors or missing imports.
    Stacktrace:
     [1] [0m[1msolve_random_distrib[22m[0m[1m([22m[90mchunk[39m::[0mUnitRange[90m{Int64}[39m, [90mf[39m::[0mtypeof(f), [90mop_list[39m::[0mVector[90m{Any}[39m, [90mcompute_t_evolution[39m::[0mBool, [90mN[39m::[0mInt64, [90mn[39m::[0mFloat64, [90md0_lb[39m::[0mFloat64, [90mnbr_max_iteration[39m::[0mInt64[0m[1m)[22m
    [90m   @[39m [32mMain[39m [90m.\[39m[90m[4mIn[17]:38[24m[39m
     [2] [0m[1m(::var"#92#93"{UnitRange{Int64}})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @[39m [32mMain[39m [90m.\[39m[90m[4mIn[21]:5[24m[39m

In [ ]:
# @save "solutions/sol_N_$(N)_r_$(r)" popup_ss popup_t nbr_error_ss nbr_error_t

In [ ]:
# @load "solutions/sol_N_$(N)_r_$(r)" popup_ss popup_t nbr_error_ss nbr_error_t

# Plots

In [ ]:
fig, axes = subplots(1, 2, figsize = (10, 5))
for i in 1:length(popup_ss)
    line, = axes[1].plot(T, popup_t[i])
    axes[1].hlines(popup_ss[i], T[1], T[end], linestyle="--", color = line.get_color())
end
axes[1].set_xlabel(L"$\gamma t$")
axes[1].set_ylabel(L"$\langle  n_{\uparrow} \rangle $")

# axes[2].scatter([i for i in 1:r if i ∉ nbr_error_ss], popup_ss)
# axes[2].hlines(mean(popup_ss), 1, r, color="black")
axes[2].set_xlabel(L"r")
axes[2].set_ylabel(L"$\langle  n_{\uparrow}^\infty \rangle$") 
print(std(popup_ss))


suptitle("N = $N, r = $r, Starting from "*L"$|\downarrow \downarrow \rangle $")
pygui(false);
# pygui(true); show()

# Brouillons

In [ ]:
                # @warn "Solve failed with retcode = $(sol.retcode). Retrying…"
#                 phi_array_f, theta_array_f = zeros(N), ones(N)*7π/8 # We start from all the atoms in the GS
# uf = u0_CFunction(phi_array_f, theta_array_f, op_list)
# sol_tasks[1][5][1][end]




        # nbr_iteration = 2
        # while ~SciMLBase.successful_retcode(sol_ss) | nbr_iteration < nbr_max_iteration
        #     phi_array_f, theta_array_f = zeros(N), ones(N)*θ_f[nbr_iteration] # We start from all the atoms in the GS
        #     uf = u0_CFunction(phi_array_f, theta_array_f, op_list)
        #     # print(θ_f[nbr_iteration])
        #     prob_ss = SteadyStateProblem(fsolve, uf)
        #     # sol_ss = solve(prob_ss, SSRootfind(); abstol=1e-3, reltol=1e-3)
        #     sol_ss = solve(prob_ss, DynamicSS(); abstol=1e-6, reltol=1e-6, maxiters=1_000_000)
        #     nbr_iteration += 1
        # end


        # print(SciMLBase.successful_retcode(sol_ss))
        # push!(popup_ss, sum(real(sol_ss[1:N])))

        # if ~SciMLBase.successful_retcode(sol_ss)
        #     push!(nbr_error_ss, i)
        # end


#         """ Return the minimum distance of a distribution of atoms r0 """
# function avg_r0(r0)
#     N = length(r0)
#     d0 = zeros(N, N) # Repetiton, atom i, distance from atom j
#     for j in 1:N
#         for k = 1:N
#             d0[j, k] = norm(r0[j]-r0[k])
#         end
#     end
#     return mean(d0)*N/(N-1)
# end




# """ Reconstruct the position of atoms """
# function reconstruct_img_distrib(N, i)
#     @load "r0/r0_N_$(N)_r_$i.jdl2" r0 L
#     plt.close("all")
#     fig = plt.figure()
#     ax = fig.add_subplot(projection="3d")
#     ax.scatter([r[1] for r in r0], [r[2] for r in r0], [r[3] for r in r0])
#     ax.set_xlabel(L"x/$\lambda$")
#     ax.set_ylabel(L"y/$\lambda$")
#     ax.set_zlabel(L"z/$\lambda$")
#     ax.set_xlim(0, L), ax.set_ylim(0, L), ax.set_zlim(0, L)
#     plt.savefig("Images_distribution/IImages_distribution_N_$(N)_r_$i")
# end